In [ ]:
# import sys
# !{sys.executable} -m pip install pip earthengine-api
# !{sys.executable} -m pip install pip geemap[all]
# !{sys.executable} -m pip install pip rasterstats 

In [1]:
import ee
# ee.Authenticate()

In [2]:
ee.Initialize()

In [3]:
import numpy as np
import requests
import os
import pandas as pd
import rasterio
import boto3
import geopandas as gpd
import io
from rasterstats import zonal_stats
import fiona
import rasterio.mask
import geemap
import glob
import boto3

In [4]:
# hide warnings
import warnings
warnings.filterwarnings('ignore')

# Read input data

In [5]:
# define directory
out_dir = os.getcwd()
aws_s3_dir = "https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data"

In [35]:
# get list of c4f cities
boundary_georef = pd.read_csv('https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary_georef.csv')
boundary_georef

,city_name,geo_name,aoi_boundary_name,units_boundary_name,city_boundary_name,country_code,geo_level
0,Salvador,BRA-Salvador,ADM4union,ADM4,BRA-Salvador-ADM4,BRA,ADM4
1,Bukavu,COD-Bukavu,ADM3union,ADM3,COD-Bukavu-ADM3,COD,ADM3
2,Uvira,COD-Uvira,ADM3union,ADM3,COD-Uvira-ADM3,COD,ADM3
3,Brazzaville,COG-Brazzaville,ADM4union,ADM4,COG-Brazzaville-ADM4,COG,ADM4
4,Barranquilla,COL-Barranquilla,ADM4union,ADM4,COL-Barranquilla-ADM4,COL,ADM4
5,Addis_Ababa,ETH-Addis_Ababa,ADM4union,ADM4,ETH-Addis_Ababa-ADM4,ETH,ADM4
6,Dire_Dawa,ETH-Dire_Dawa,ADM3union,ADM3,ETH-Dire_Dawa-ADM3,ETH,ADM3
7,Nairobi,KEN-Nairobi,ADM3union,ADM3,KEN-Nairobi-ADM3,KEN,ADM3
8,Antananarivo,MDG-Antananarivo,ADM4union,ADM4,MDG-Antananarivo-ADM4,MDG,ADM4
9,Mexico_City,MEX-Mexico_City,ADM2union,ADM2,MEX-Mexico_City-ADM2,MEX,ADM2


In [36]:
# remove cities with time out issues due to volumetry
data_volumetry_error_cities = ['MEX-Mexico_City']
boundary_georef = boundary_georef[~boundary_georef['geo_name'].isin(data_volumetry_error_cities)].reset_index(drop=True)
boundary_georef

,city_name,geo_name,aoi_boundary_name,units_boundary_name,city_boundary_name,country_code,geo_level
0,Salvador,BRA-Salvador,ADM4union,ADM4,BRA-Salvador-ADM4,BRA,ADM4
1,Bukavu,COD-Bukavu,ADM3union,ADM3,COD-Bukavu-ADM3,COD,ADM3
2,Uvira,COD-Uvira,ADM3union,ADM3,COD-Uvira-ADM3,COD,ADM3
3,Brazzaville,COG-Brazzaville,ADM4union,ADM4,COG-Brazzaville-ADM4,COG,ADM4
4,Barranquilla,COL-Barranquilla,ADM4union,ADM4,COL-Barranquilla-ADM4,COL,ADM4
5,Addis_Ababa,ETH-Addis_Ababa,ADM4union,ADM4,ETH-Addis_Ababa-ADM4,ETH,ADM4
6,Dire_Dawa,ETH-Dire_Dawa,ADM3union,ADM3,ETH-Dire_Dawa-ADM3,ETH,ADM3
7,Nairobi,KEN-Nairobi,ADM3union,ADM3,KEN-Nairobi-ADM3,KEN,ADM3
8,Antananarivo,MDG-Antananarivo,ADM4union,ADM4,MDG-Antananarivo-ADM4,MDG,ADM4
9,Monterrey,MEX-Monterrey,ADM2union,ADM2,MEX-Monterrey-ADM2,MEX,ADM2


# Compute indicator

In [37]:
cities_indicators_GRE_3_23 = pd.DataFrame() 

In [38]:
for i in range(0,len(boundary_georef)):
    print(i)
    geo_name = boundary_georef.loc[i, 'geo_name']
    print("\n geo_name: "+geo_name)
    
    boundary_id_aoi = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    boundary_id_unit = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'units_boundary_name']

    # process aoi level ------
    print("\n boundary_id_aoi: "+boundary_id_aoi)
    # read boundaries
    boundary_path = aws_s3_dir +'/boundaries/v_0/boundary-'+boundary_id_aoi+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    
    #read open space
    openspace_path = 'https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data/open_space/openstreetmap/v_0/'+boundary_id_aoi+'-OSM-open_space-2022.geojson'
    openspace_geo = requests.get(openspace_path).json()
    openspace_geo_ee = geemap.geojson_to_ee(openspace_geo)
    
    #load population
    pop = ee.ImageCollection('WorldPop/GP/100m/pop')
    pop = (pop.filter(ee.Filter.And(
        ee.Filter.bounds(boundary_geo_ee),
        ee.Filter.inList('year',[2020])))
        .select('population'))
    popImg = pop.mean().rename('population')

    popProj = pop.first().projection()
    popScale = popProj.nominalScale()

    # define threshold distance (m) and buffer open space areas by that to get recreation catchment
    DistanceThres = 400 # meters distance from population to be considered
    def amenityBuffer(feat):
      feat = ee.Feature(feat)
      return feat.buffer(DistanceThres)
    RecCatchment = openspace_geo_ee.map(amenityBuffer)
    RecCatchmentUnion = RecCatchment.union()
    
    #mask population by recreation catchment
    mask = ee.Image.constant(1).clip(RecCatchmentUnion.geometry()).mask()
    popwOSaccess = popImg.updateMask(mask).rename('populationwOpenSpace')
    
    ## add tree cover dataset
    TML = ee.ImageCollection('projects/wri-datalab/TML')
    TreeCover = TML.reduce(ee.Reducer.mean()).rename('b1')
    
    # calcs for % population with threshold level (e.g. 10%+) of tree cover within walking distance (e.g. 400m)
    TreePctThreshold = 10 #whole numbers - 0-100, minimum percentage of tree cover threshold to consider 
    circleTheshm = ee.Kernel.circle(DistanceThres, 'meters', False)
    TreeCoverinThreshm = TreeCover.reduceNeighborhood(ee.Reducer.mean(), circleTheshm)
    popwthresTC = popImg.updateMask(TreeCoverinThreshm.gte(TreePctThreshold)).rename('populationwTreeCover')
    
    # combine images 
    combImg = popImg.addBands([popwOSaccess,popwthresTC])
    
    # function to calculate indicators
    def calcs(feat):
     return feat.set({
        'PopwOpenSpaceAccessPct': feat.getNumber('populationwOpenSpace').divide(feat.getNumber('population')),
        'PopwTreeCoverAccessPct': feat.getNumber('populationwTreeCover').divide(feat.getNumber('population'))
     }) 

    # use sum reducer to get total populations by features 
    PopbyDistrict=combImg.reduceRegions(
      reducer= ee.Reducer.sum(), 
      collection= boundary_geo_ee, 
      scale= popScale, 
      tileScale= 4
    )

    # # apply function
    PopbyDistrict = PopbyDistrict.map(calcs).select(['geo_id','PopwOpenSpaceAccessPct','PopwTreeCoverAccessPct'])
    
    # store in df and apend
    df = geemap.ee_to_pandas(PopbyDistrict)#,['geo_id','PopwOpenSpaceAccessPct'])
    df = df.rename(columns={"PopwOpenSpaceAccessPct": "GRE_3_2_percentPopwOpenSpaceAccess"}).rename(columns={"PopwTreeCoverAccessPct": "GRE_3_3_percentPopwTreeCoverAcess"})
    cities_indicators_GRE_3_23 = cities_indicators_GRE_3_23.append(df)
    
    # process unit of analysis level ------
    print("\n boundary_id_unit: "+boundary_id_unit)
    # read boundaries
    boundary_path = aws_s3_dir +'/boundaries/v_0/boundary-'+boundary_id_unit+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    
    # use sum reducer to get total populations by features 
    PopbyDistrict=combImg.reduceRegions(
      reducer= ee.Reducer.sum(), 
      collection= boundary_geo_ee, 
      scale= popScale, 
      tileScale= 4
    )
    
    # apply function
    PopbyDistrict = PopbyDistrict.map(calcs).select(['geo_id','PopwOpenSpaceAccessPct','PopwTreeCoverAccessPct'])
    
    # store in df and apend
    df = geemap.ee_to_pandas(PopbyDistrict)
    df = df.rename(columns={"PopwOpenSpaceAccessPct": "GRE_3_2_percentPopwOpenSpaceAccess"}).rename(columns={"PopwTreeCoverAccessPct": "GRE_3_3_percentPopwTreeCoverAcess"})
    cities_indicators_GRE_3_23 = cities_indicators_GRE_3_23.append(df)
    

0

 geo_name: BRA-Salvador

 boundary_id_aoi: BRA-Salvador-ADM4union

 boundary_id_unit: BRA-Salvador-ADM4
1

 geo_name: COD-Bukavu

 boundary_id_aoi: COD-Bukavu-ADM3union

 boundary_id_unit: COD-Bukavu-ADM3
2

 geo_name: COD-Uvira

 boundary_id_aoi: COD-Uvira-ADM3union

 boundary_id_unit: COD-Uvira-ADM3
3

 geo_name: COG-Brazzaville

 boundary_id_aoi: COG-Brazzaville-ADM4union

 boundary_id_unit: COG-Brazzaville-ADM4
4

 geo_name: COL-Barranquilla

 boundary_id_aoi: COL-Barranquilla-ADM4union

 boundary_id_unit: COL-Barranquilla-ADM4
5

 geo_name: ETH-Addis_Ababa

 boundary_id_aoi: ETH-Addis_Ababa-ADM4union

 boundary_id_unit: ETH-Addis_Ababa-ADM4
6

 geo_name: ETH-Dire_Dawa

 boundary_id_aoi: ETH-Dire_Dawa-ADM3union

 boundary_id_unit: ETH-Dire_Dawa-ADM3
7

 geo_name: KEN-Nairobi

 boundary_id_aoi: KEN-Nairobi-ADM3union

 boundary_id_unit: KEN-Nairobi-ADM3
8

 geo_name: MDG-Antananarivo

 boundary_id_aoi: MDG-Antananarivo-ADM4union

 boundary_id_unit: MDG-Antananarivo-ADM4
9

 geo_na

In [39]:
cities_indicators_GRE_3_23

,GRE_3_3_percentPopwTreeCoverAcess,GRE_3_2_percentPopwOpenSpaceAccess,geo_id
0,0.004452,0.743757,BRA-Salvador_ADM4-union_1
0,0.000000,0.990298,BRA-Salvador_ADM4_1
1,0.000000,0.984178,BRA-Salvador_ADM4_2
2,0.000000,0.936113,BRA-Salvador_ADM4_3
3,0.000000,0.945183,BRA-Salvador_ADM4_4
...,...,...,...
135,0.953596,0.000000,RWA-Musanze_ADM5_136
136,0.000000,0.000000,RWA-Musanze_ADM5_137
137,0.000000,0.000000,RWA-Musanze_ADM5_138
138,0.000000,0.000000,RWA-Musanze_ADM5_139


In [51]:
cities_indicators_GRE_3_2 = cities_indicators_GRE_3_23[["geo_id","GRE_3_2_percentPopwOpenSpaceAccess"]]
cities_indicators_GRE_3_3 = cities_indicators_GRE_3_23[["geo_id","GRE_3_3_percentPopwTreeCoverAcess"]]

# Merge with indicator table

In [52]:
# read indicator table
#cities_indicators = pd.read_csv(aws_s3_dir + '/indicators/cities_indicators_erictest.csv')
cities_indicators = pd.read_csv(aws_s3_dir + '/indicators/cities_indicators_v2.csv')
cities_indicators

,geo_id,geo_level,geo_name,geo_parent_name,percent_of_tree_cover,GRE_3_1_percentOpenSpaceinBuiltup
0,BRA-Salvador_ADM4-union_1,ADM4-union,BRA-Salvador,BRA-Salvador,NaN,0.043743
1,BRA-Salvador_ADM4_1,ADM4,Pituaçu,BRA-Salvador,NaN,0.072329
2,BRA-Salvador_ADM4_2,ADM4,Patamares,BRA-Salvador,NaN,0.103440
3,BRA-Salvador_ADM4_3,ADM4,Piatã,BRA-Salvador,NaN,0.050375
4,BRA-Salvador_ADM4_4,ADM4,Boca do Rio,BRA-Salvador,NaN,0.048150
...,...,...,...,...,...,...
1078,RWA-Musanze_ADM5_136,ADM5,Burengo,RWA-Musanze,28.526825,0.000000
1079,RWA-Musanze_ADM5_137,ADM5,Bwamazi,RWA-Musanze,NaN,0.000000
1080,RWA-Musanze_ADM5_138,ADM5,Kadahenda,RWA-Musanze,NaN,0.000000
1081,RWA-Musanze_ADM5_139,ADM5,Karwesero,RWA-Musanze,NaN,0.000000


In [53]:
def merge_indicators(indicator_table, new_indicator_table, indicator_name):
    if indicator_name in indicator_table.columns:
        print("replace by new indicators")
        indicator_table.drop(indicator_name, inplace=True, axis=1)
        cities_indicators_df = indicator_table.merge(new_indicator_table[["geo_id",indicator_name]], 
                                                     on='geo_id', 
                                                     how='left')
    else:
        print("add new indicators")
        cities_indicators_df = indicator_table.merge(new_indicator_table[["geo_id",indicator_name]], 
                                                     on='geo_id', 
                                                     how='left')
    return(cities_indicators_df)

In [58]:
cities_indicators_merged = merge_indicators(indicator_table = cities_indicators,
                                            new_indicator_table = cities_indicators_GRE_3_2,
                                            indicator_name = "GRE_3_2_percentPopwOpenSpaceAccess")

add new indicators


In [59]:
cities_indicators_merged = merge_indicators(indicator_table = cities_indicators_merged,
                                            new_indicator_table = cities_indicators_GRE_3_3,
                                            indicator_name = "GRE_3_3_percentPopwTreeCoverAcess")

add new indicators


In [60]:
cities_indicators_merged

,geo_id,geo_level,geo_name,geo_parent_name,percent_of_tree_cover,GRE_3_1_percentOpenSpaceinBuiltup,GRE_3_2_percentPopwOpenSpaceAccess,GRE_3_3_percentPopwTreeCoverAcess
0,BRA-Salvador_ADM4-union_1,ADM4-union,BRA-Salvador,BRA-Salvador,NaN,0.043743,0.743757,0.004452
1,BRA-Salvador_ADM4_1,ADM4,Pituaçu,BRA-Salvador,NaN,0.072329,0.990298,0.000000
2,BRA-Salvador_ADM4_2,ADM4,Patamares,BRA-Salvador,NaN,0.103440,0.984178,0.000000
3,BRA-Salvador_ADM4_3,ADM4,Piatã,BRA-Salvador,NaN,0.050375,0.936113,0.000000
4,BRA-Salvador_ADM4_4,ADM4,Boca do Rio,BRA-Salvador,NaN,0.048150,0.945183,0.000000
...,...,...,...,...,...,...,...,...
1078,RWA-Musanze_ADM5_136,ADM5,Burengo,RWA-Musanze,28.526825,0.000000,0.000000,0.953596
1079,RWA-Musanze_ADM5_137,ADM5,Bwamazi,RWA-Musanze,NaN,0.000000,0.000000,0.000000
1080,RWA-Musanze_ADM5_138,ADM5,Kadahenda,RWA-Musanze,NaN,0.000000,0.000000,0.000000
1081,RWA-Musanze_ADM5_139,ADM5,Karwesero,RWA-Musanze,NaN,0.000000,0.000000,0.000000


# Upload in aws s3

In [61]:
# connect to s3
#aws_credentials = pd.read_csv('/home/jovyan/PlanetaryComputerExamples/aws_credentials.csv')
aws_credentials = pd.read_csv('C:\\Users\\Saif.Shabou\\OneDrive - World Resources Institute\\Documents\\aws\\credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

In [62]:
# upload to aws
# key_data = 'data/indicators/cities_indicators_erictest.csv'
key_data = 'data/indicators/cities_indicators_v2.csv'
bucket_name = 'cities-cities4forests' 
cities_indicators_merged.to_csv(
    f"s3://{bucket_name}/{key_data}",
    index=False,
    storage_options={
        "key": aws_key,
        "secret": aws_secret
    },
)

In [63]:
# make it public
object_acl = s3.ObjectAcl(bucket_name,key_data)
response = object_acl.put(ACL='public-read')